In [1]:
# This script is for trainit the model.
# For base ve use pretrained model cjvt/gpt-sl-base
# https://huggingface.co/cjvt/gpt-sl-base

In [2]:
!pip install transformers 
!pip install --upgrade accelerate
!pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
import pandas as pd
import os
from datetime import datetime as dt

from transformers import (
    AutoModelWithLMHead, 
    AutoModelForCausalLM,
    AutoConfig,
    Trainer,
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments)

In [4]:
# if the variable is set to true, then the script takes the base model (cjvt/t5-sl-small)
# If the variable is set to False, then searches in direcotry /content/gdrive/MyDrive/NLPProject/models/T5 for latest trained model
newModel = False

In [5]:
# Reading Excel file with the sentences we are using to train the model
drive.mount('/content/gdrive/', force_remount=True)
trainDatabase = pd.read_excel("/content/gdrive/MyDrive/NLPProject/data/DeeplTrening.xlsx")

Mounted at /content/gdrive/


In [6]:
# Preparing data
trainDatabase["parafraziranje"] = '<s>'+trainDatabase['parafraza']+'</s>'+'>>>>'+'<p>'+trainDatabase['original']+'</p>'
trainDatabase = trainDatabase.sample(n = 5000)
trainDatabase['parafraziranje'] = trainDatabase.parafraziranje.to_csv('parafraziranje.txt',sep='\n',index=False)

In [7]:
# Seraching for path to latest build model
pot = "/content/gdrive/MyDrive/NLPProject/models/GPT/"

pathToModel = ""
cas = 0

# Pregleda kateri model je zadnji
for filename in os.listdir(pot):
    if os.path.isdir(os.path.join(pot,filename)):
      if os.path.getmtime(os.path.join(pot,filename)) > cas:
        cas = os.path.getmtime(os.path.join(pot,filename))
        pathToModel = os.path.join(pot,filename)

print(pathToModel,cas)

/content/gdrive/MyDrive/NLPProject/models/GPT/2023-05-23T18:49:50.480095 1684867903.0


In [8]:
# Seting the destination folder for model
# The name of the directory is made in such a way that the date on which the model was made is clear

potZaModel = "/content/gdrive/MyDrive/NLPProject/models/GPT/"+dt.now().isoformat()
print(potZaModel)

/content/gdrive/MyDrive/NLPProject/models/GPT/2023-05-23T18:53:38.237076


In [9]:
def modelTrainer(text_path, epochs, modell='cjvt/gpt-sl-base', batch_size=8):

  print(modell)

  model = AutoModelForCausalLM.from_pretrained(modell)	
  tokenizer = AutoTokenizer.from_pretrained('cjvt/gpt-sl-base')
  tokenizer.model_max_length = 50
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

  train_dataset = TextDataset(
      tokenizer=tokenizer, 
      file_path = text_path, 
      block_size=256
  )

  print(type(train_dataset))
  trainig_args = TrainingArguments(
      output_dir=potZaModel,
      num_train_epochs = epochs,
      per_device_train_batch_size = batch_size,
      warmup_steps = 500,
      save_steps = 5000,
      logging_steps = 10,
      optim='adamw_torch',
  )

  trainer = Trainer(
		model=model,
		args = trainig_args,
		data_collator = data_collator,
		train_dataset = train_dataset,
    #cache_enabled = False,
		#prediction_loss_only = True
  )

  trainer.train()
  trainer.save_model()

In [10]:
# Loading model for training and training

if newModel:
  print("Training a new model.")
  modelTrainer("parafraziranje.txt", epochs=1, batch_size=8)
else:
  print("Training existing model")
  modelTrainer("parafraziranje.txt", epochs=1, batch_size=8, modell = pathToModel)

Training existing model
/content/gdrive/MyDrive/NLPProject/models/GPT/2023-05-23T18:49:50.480095


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (306910 > 50). Running this sequence through the model will result in indexing errors


<class 'transformers.data.datasets.language_modeling.TextDataset'>


Step,Training Loss
10,1.734200
20,1.745500
30,1.718300
40,1.770300
50,1.705000
60,1.697300
70,1.688200
80,1.732200
90,1.733000
100,1.722400


In [11]:
# Removing temporary files

!rm -rf cac*
!rm -rf *.txt